# Purpose

### 2021-10-25
In this notebook I'll select the clusters for the One Feed experiment for DE to DE subreddits.

From manual inspection on mlflow GUI the best candidate is:<br>
`134cefe13ae34621a69fcc48c4d5fb71`

Because:
- it has high scores at the 100-to-200 & 200-to-300 bins 
- AND has the most subreddits (filtered out fewer subreddits due to low post counts)

Other clusters had slightly higher values at the 200-to-300 bin, but they clustered fewer subreddits.

**NOTE | 2021-11-10**<br>
This table should already be in BigQuery with standardized names, so read it from there rather than from GCS.

# Imports & notebook setup

In [1]:
%load_ext google.colab.data_table

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# colab auth for BigQuery
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [4]:
# Attach google drive & import my python utility functions
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

import sys
l_paths_to_append = [
    '/content/gdrive/MyDrive/Colab Notebooks',

    # need to append the path to subclu so that colab can import things properly
    '/content/gdrive/MyDrive/Colab Notebooks/subreddit_clustering_i18n'
]
for path_ in l_paths_to_append:
    if not path_ in sys.path:
        print(f"Appending: {path_}")
        sys.path.append(path_)

Mounted at /content/gdrive
Appending: /content/gdrive/MyDrive/Colab Notebooks


In [5]:
## install subclu & libraries needed to read parquet files from GCS

!pip install -e "/content/gdrive/MyDrive/Colab Notebooks/subreddit_clustering_i18n/" --quiet

In [6]:
# Install needed to load data from GCS, for some reason not included in subclu?
!pip install gcsfs --quiet

In [60]:
# Need to get latest version of gspread to read & write to google sheets
!pip install --upgrade gspread --quiet

In [8]:
# auth for google sheets
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [9]:
# Regular Imports
import os
from datetime import datetime

from google.cloud import bigquery

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib_venn import venn2_unweighted, venn3_unweighted


# os.environ['GOOGLE_CLOUD_PROJECT'] = 'data-science-prod-218515'
os.environ['GOOGLE_CLOUD_PROJECT'] = 'data-prod-165221'

In [10]:
# subclu imports

# For reloading, need to force-delete some imported items
try:
    del LoadPosts, LoadSubreddits
except Exception:
    pass

from subclu.utils.hydra_config_loader import LoadHydraConfig
from subclu.data.data_loaders import LoadPosts, LoadSubreddits
from subclu.utils.eda import (
    setup_logging, counts_describe, value_counts_and_pcts,
    notebook_display_config, print_lib_versions,
    style_df_numeric, reorder_array,
)


setup_logging()
print_lib_versions([pd, np])

python		v 3.7.12
===
pandas		v: 1.1.5
numpy		v: 1.19.5


# Load subreddit metadata

This data is already in bigQuery so read it straight from there. We'll use it to filter out geo-relevant (German) subs.

Also add the latest ratings so that we can filter based on those.

## SQL query

In [11]:
%%time

sql_geo_and_languages = f"""
-- select DE subreddits + get latest rating

SELECT 
    sl.subreddit_id
    , sl.subreddit_name
    , r.rating
    -- , r.subrating
    , r.version

    , slo.verdict
    , slo.quarantine

    , geo.country_name
    , geo.users_percent_in_country
    -- , sl.geo_relevant_countries
    , ambassador_subreddit
    , posts_for_modeling_count

    , primary_post_language
    , primary_post_language_percent
    , secondary_post_language
    , secondary_post_language_percent

    , geo_relevant_country_count
    , geo_relevant_country_codes
    , geo_relevant_subreddit

FROM `reddit-employee-datasets.david_bermejo.subclu_v0040_subreddit_languages` sl
LEFT JOIN (
    SELECT * FROM `data-prod-165221.ds_v2_postgres_tables.subreddit_lookup`
    # Look back 2 days because looking back 1-day could be an empty partition
    WHERE dt = (CURRENT_DATE() - 2)
) AS slo
    ON slo.subreddit_id = sl.subreddit_id
LEFT JOIN (
    SELECT * FROM `reddit-employee-datasets.david_bermejo.subclu_geo_subreddits_20210922`
    WHERE country_name = 'Germany'
) AS geo 
    ON sl.subreddit_id = geo.subreddit_id
LEFT JOIN (
    SELECT * FROM ds_v2_subreddit_tables.subreddit_ratings
    WHERE DATE(pt) = (CURRENT_DATE() - 2)
) AS r
    ON r.subreddit_id = sl.subreddit_id

    
WHERE 1=1
    -- AND r.version = 'v2'
    -- AND COALESCE(r.rating, '') IN ('pg', 'pg13', 'g')
    AND COALESCE(slo.verdict, '') != 'admin-removed'
    AND COALESCE(slo.quarantine, false) != true
    AND (
        sl.geo_relevant_countries LIKE '%Germany%'
        OR sl.ambassador_subreddit = True
    )

ORDER BY users_percent_in_country ASC -- subreddit_name, ambassador_subreddit
;
"""

client = bigquery.Client()
df_geo_and_lang = client.query(sql_geo_and_languages).to_dataframe()
print(df_geo_and_lang.shape)

(827, 17)
CPU times: user 124 ms, sys: 17.2 ms, total: 141 ms
Wall time: 9.42 s


## Check data with geo + language information

In [12]:
df_geo_and_lang.head()

,subreddit_id,subreddit_name,rating,version,verdict,quarantine,country_name,users_percent_in_country,ambassador_subreddit,posts_for_modeling_count,primary_post_language,primary_post_language_percent,secondary_post_language,secondary_post_language_percent,geo_relevant_country_count,geo_relevant_country_codes,geo_relevant_subreddit
0,t5_4ckovw,buehne,None,None,None,False,None,NaN,True,9.0,German,0.333333,Danish,0.111111,NaN,None,False
1,t5_4p0iav,de_events,None,None,None,False,None,NaN,True,1.0,German,1.000000,None,NaN,NaN,None,False
2,t5_2otu32,nikolacorporation,pg,v3,None,False,Germany,0.160008,False,188.0,English,0.920213,Estonian,0.010638,1.0,DE,True
3,t5_vwvbb,vanmoofbicycle,pg,v1,None,False,Germany,0.160199,False,305.0,English,0.963934,None,NaN,1.0,DE,True
4,t5_2rq3g,trackmania,pg,v1,None,False,Germany,0.160615,False,958.0,English,0.927975,None,NaN,1.0,DE,True


In [13]:
df_geo_and_lang.tail()

,subreddit_id,subreddit_name,rating,version,verdict,quarantine,country_name,users_percent_in_country,ambassador_subreddit,posts_for_modeling_count,primary_post_language,primary_post_language_percent,secondary_post_language,secondary_post_language_percent,geo_relevant_country_count,geo_relevant_country_codes,geo_relevant_subreddit
822,t5_2tz7b,braunschweig,None,None,None,False,Germany,0.956430,False,28.0,German,0.892857,English,0.071429,1.0,DE,True
823,t5_3255n,duschgedanken,None,None,None,False,Germany,0.956434,True,90.0,German,1.000000,None,NaN,1.0,DE,True
824,t5_2w4vt,bielefeld,None,None,None,False,Germany,0.957314,False,27.0,German,0.925926,English,0.037037,1.0,DE,True
825,t5_2ty5z,bundeswehr,None,None,None,False,Germany,0.959926,False,254.0,German,0.960630,English,0.031496,1.0,DE,True
826,t5_4o0ba2,nachthimmel,None,None,None,False,Germany,1.000000,True,18.0,German,0.833333,English,0.111111,NaN,None,False


# Load model labels

~Ideally we could just pull the configuration data from github~

The clusters now live in a big Query table and have standardized names, so pull the data from there.

## Pull data from BigQuery


In [14]:
%%time

sql_labels = f"""
-- select subreddit clusters from bigQuery

SELECT
    subreddit_id
    , subreddit_name
    , sc.* EXCEPT(subreddit_id, subreddit_name)
FROM `reddit-employee-datasets.david_bermejo.subclu_v0040_subreddit_clusters_c_a` sc
;
"""

client = bigquery.Client()
df_labels = client.query(sql_labels).to_dataframe()
print(df_labels.shape)

(19053, 17)
CPU times: user 931 ms, sys: 76.7 ms, total: 1.01 s
Wall time: 3.33 s


In [15]:
df_labels.head()

,subreddit_id,subreddit_name,model_distance_order,posts_for_modeling_count,primary_topic_0921,k052_label,k100_label,k248_label,k351_label,k405_label,k052_majority_primary_topic,k100_majority_primary_topic,k248_majority_primary_topic,k351_majority_primary_topic,k405_majority_primary_topic,table_creation_date,mlflow_run_uuid
0,t5_386fy,xtrill,41,32,None,1,1,1,1,1,Music,Music,Music,Music,Music,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71
1,t5_2roop,hardtechno,49,18,None,1,1,1,1,1,Music,Music,Music,Music,Music,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71
2,t5_2qziu,rappers,76,386,None,1,1,1,1,1,Music,Music,Music,Music,Music,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71
3,t5_2vrp8,musicaargentina,84,26,None,1,1,1,1,1,Music,Music,Music,Music,Music,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71
4,t5_2smd3,musik,85,23,None,1,1,1,1,1,Music,Music,Music,Music,Music,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71


In [16]:
counts_describe(df_labels)

,dtype,count,unique,unique-percent,null-count,null-percent
subreddit_id,object,"19,053","19,053",100.00%,0,0.00%
subreddit_name,object,"19,053","19,053",100.00%,0,0.00%
model_distance_order,int64,"19,053","19,053",100.00%,0,0.00%
posts_for_modeling_count,int64,"19,053","1,175",6.17%,0,0.00%
primary_topic_0921,object,"15,929",51,0.32%,"3,124",16.40%
k052_label,int64,"19,053",52,0.27%,0,0.00%
k100_label,int64,"19,053",100,0.52%,0,0.00%
k248_label,int64,"19,053",248,1.30%,0,0.00%
k351_label,int64,"19,053",351,1.84%,0,0.00%
k405_label,int64,"19,053",405,2.13%,0,0.00%


# Keep only labels for DE subreddits


In [35]:
l_ix_subs = ['subreddit_name', 'subreddit_id']

df_labels_de = (
    df_labels
    .merge(
        df_geo_and_lang.drop(['posts_for_modeling_count'], axis=1),
        how='right',
        on=l_ix_subs,
    )
    .copy()
    .sort_values(by=['model_distance_order'], ascending=True)
)

# move some columns to the end of the file
l_cols_to_end = ['table_creation_date', 'mlflow_run_uuid']

df_labels_de = df_labels_de[
    df_labels_de.drop(l_cols_to_end, axis=1).columns.to_list() +
    l_cols_to_end
]
print(df_labels_de.shape)

(827, 31)


### Drop subs with too few posts

In the modeling process I drop subreddits with too few posts. We don't have recommendations for them, so let's drop them.

It would also not be a great experience to recommend dead subs.

In [36]:
df_labels_de['model_distance_order'].isnull().sum()

33

In [37]:
df_labels_de = df_labels_de[
    ~df_labels_de['model_distance_order'].isnull()
].copy()
df_labels_de['model_distance_order'] = df_labels_de['model_distance_order'].astype(int)

l_cols_label_de = [c for c in df_labels_de.columns if c.endswith('_label')]
df_labels_de[l_cols_label_de] = df_labels_de[l_cols_label_de].astype(int)

df_labels_de.shape

(794, 31)

In [38]:
style_df_numeric(
    df_labels_de.head(15),
    # rename_cols_for_display=True,
    pct_labels=['_percent_in_country', '_percent'],
    int_labels=None,
    pct_cols=['users_percent_in_country'],
    l_bar_simple=[c for c in df_labels_de.columns if '_label' in c]
)

,subreddit_id,subreddit_name,model_distance_order,posts_for_modeling_count,primary_topic_0921,k052_label,k100_label,k248_label,k351_label,k405_label,k052_majority_primary_topic,k100_majority_primary_topic,k248_majority_primary_topic,k351_majority_primary_topic,k405_majority_primary_topic,rating,version,verdict,quarantine,country_name,users_percent_in_country,ambassador_subreddit,primary_post_language,primary_post_language_percent,secondary_post_language,secondary_post_language_percent,geo_relevant_country_count,geo_relevant_country_codes,geo_relevant_subreddit,table_creation_date,mlflow_run_uuid
33,t5_2roop,hardtechno,49,18,None,1,1,1,1,1,Music,Music,Music,Music,Music,None,None,None,False,Germany,0,False,English,77.78%,Dutch,5.56%,1,DE,True,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71
151,t5_2qziu,rappers,76,386,None,1,1,1,1,1,Music,Music,Music,Music,Music,r,v2,admin-approved,False,Germany,0,False,English,81.87%,German,1.81%,1,DE,True,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71
575,t5_2v7pv,germanrap,77,705,Music,1,1,1,1,1,Music,Music,Music,Music,Music,r,v2,None,False,Germany,1,False,German,71.49%,English,13.05%,1,DE,True,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71
229,t5_2smd3,musik,85,23,None,1,1,1,1,1,Music,Music,Music,Music,Music,None,None,None,False,Germany,1,False,German,82.61%,English,8.70%,1,DE,True,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71
199,t5_39ea8,mgpmppjwfa,89,25,Music,1,1,1,1,1,Music,Music,Music,Music,Music,None,None,None,False,Germany,0,False,German,52.00%,English,8.00%,1,DE,True,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71
243,t5_2t6i4,germusic,95,179,Music,1,1,1,1,1,Music,Music,Music,Music,Music,pg13,None,None,False,Germany,1,False,German,58.10%,English,21.23%,1,DE,True,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71
546,t5_31l12,kollegah,172,17,Music,1,2,2,2,2,Music,Music,Music,Music,Music,None,None,None,False,Germany,1,False,German,94.12%,Indonesian,5.88%,1,DE,True,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71
318,t5_2ylk3,moneyboy,173,19,Music,1,2,2,2,2,Music,Music,Music,Music,Music,None,None,None,False,Germany,1,False,German,84.21%,Danish,5.26%,1,DE,True,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71
13,t5_2t36v,billytalent,283,101,Music,1,2,4,4,4,Music,Music,Music,Music,Music,r,v1,None,False,Germany,0,False,English,99.01%,Somali,0.99%,1,DE,True,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71
14,t5_35q0o,lindemann,370,37,Music,1,2,4,4,4,Music,Music,Music,Music,Music,r,v2,None,False,Germany,0,False,English,91.89%,Danish,2.70%,1,DE,True,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71


In [39]:
style_df_numeric(
    df_labels_de.tail(15),
    # rename_cols_for_display=True,
    l_bar_simple=[c for c in df_labels_de.columns if '_label' in c]
)

,subreddit_id,subreddit_name,model_distance_order,posts_for_modeling_count,primary_topic_0921,k052_label,k100_label,k248_label,k351_label,k405_label,k052_majority_primary_topic,k100_majority_primary_topic,k248_majority_primary_topic,k351_majority_primary_topic,k405_majority_primary_topic,rating,version,verdict,quarantine,country_name,users_percent_in_country,ambassador_subreddit,primary_post_language,primary_post_language_percent,secondary_post_language,secondary_post_language_percent,geo_relevant_country_count,geo_relevant_country_codes,geo_relevant_subreddit,table_creation_date,mlflow_run_uuid
523,t5_38yld,informatik,"18,499",17,None,51,95,239,340,392,Careers,Careers,Careers,Careers,Careers,None,None,None,False,Germany,1,False,German,100.00%,None,-,1,DE,True,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71
232,t5_2tevb,dejobs,"18,508",46,Careers,51,95,239,340,392,Careers,Careers,Careers,Careers,Careers,None,None,None,False,Germany,1,False,English,76.09%,German,15.22%,1,DE,True,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71
729,t5_34p2nx,arbeitsleben,"18,555",158,Careers,51,95,240,341,393,Careers,Careers,Careers,Careers,Careers,None,None,None,False,Germany,1,False,German,99.37%,None,-,1,DE,True,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71
129,t5_3f382,yotta,"18,821",124,"Business, Economics, and Finance",52,97,243,346,399,"Business, Economics, and Finance","Business, Economics, and Finance","Business, Economics, and Finance","Business, Economics, and Finance","Business, Economics, and Finance",None,None,None,False,Germany,0,False,English,95.97%,Croatian,0.81%,1,DE,True,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71
70,t5_3jv3m,n26bank,"18,826",36,Technology,52,97,243,346,399,"Business, Economics, and Finance","Business, Economics, and Finance","Business, Economics, and Finance","Business, Economics, and Finance","Business, Economics, and Finance",None,None,None,False,Germany,0,False,English,100.00%,None,-,1,DE,True,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71
126,t5_3bbpl,number26bank,"18,827",47,Technology,52,97,243,346,399,"Business, Economics, and Finance","Business, Economics, and Finance","Business, Economics, and Finance","Business, Economics, and Finance","Business, Economics, and Finance",pg13,v1,None,False,Germany,0,False,English,100.00%,None,-,1,DE,True,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71
513,t5_3vuqez,immobilieninvestments,"18,845",61,Learning and Education,52,98,244,347,400,"Business, Economics, and Finance",Home and Garden,"Business, Economics, and Finance","Business, Economics, and Finance","Business, Economics, and Finance",None,None,None,False,Germany,1,False,German,98.36%,English,1.64%,1,DE,True,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71
781,t5_37u60,steuern,"18,911",39,"Business, Economics, and Finance",52,99,246,349,402,"Business, Economics, and Finance","Business, Economics, and Finance","Business, Economics, and Finance","Business, Economics, and Finance","Business, Economics, and Finance",None,None,None,False,Germany,1,False,German,97.44%,Norwegian,2.56%,1,DE,True,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71
769,t5_35m5e,finanzen,"18,921","1,200","Business, Economics, and Finance",52,99,246,349,402,"Business, Economics, and Finance","Business, Economics, and Finance","Business, Economics, and Finance","Business, Economics, and Finance","Business, Economics, and Finance",pg,v1,None,False,Germany,1,False,German,97.92%,English,1.75%,1,DE,True,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71
39,t5_3ljid,europefire,"18,931",51,"Business, Economics, and Finance",52,99,246,349,402,"Business, Economics, and Finance","Business, Economics, and Finance","Business, Economics, and Finance","Business, Economics, and Finance","Business, Economics, and Finance",pg13,v2,None,False,Germany,0,False,English,100.00%,None,-,1,DE,True,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71


## Check if there are any single subreddits if we use cluster = 52

We want to avoid having clusters of one subreddit b/c that means we have nothing to recommend.

Looks like even at 52 there are some subreddits that are orphans.

In [40]:
value_counts_and_pcts(
    df_labels_de['k052_label'], top_n=10,
    reset_index=True,
    add_col_prefix=False,
)

,k052_label,count,percent,cumulative_percent
0,9,83,10.5%,10.5%
1,7,65,8.2%,18.6%
2,49,54,6.8%,25.4%
3,12,50,6.3%,31.7%
4,21,41,5.2%,36.9%
5,47,37,4.7%,41.6%
6,23,37,4.7%,46.2%
7,18,34,4.3%,50.5%
8,20,33,4.2%,54.7%
9,6,30,3.8%,58.4%


In [41]:
df_lbl_counts = value_counts_and_pcts(
    df_labels_de['k052_label'], top_n=None,
    reset_index=True,
    add_col_prefix=False,
    return_df=True,
).sort_values(by=['count'], ascending=False)

In [42]:
style_df_numeric(df_lbl_counts.tail())

,k052_label,count,percent,cumulative_percent
43,22,3,0.38%,99.24%
44,44,2,0.25%,99.50%
45,15,2,0.25%,99.75%
46,48,1,0.13%,99.87%
47,43,1,0.13%,100.00%


In [43]:
l_orphan_cluster_ids = df_lbl_counts[df_lbl_counts['count'] <= 1]['k052_label'].values

df_labels_de[df_labels_de['k052_label'].isin(l_orphan_cluster_ids)]

,subreddit_id,subreddit_name,model_distance_order,posts_for_modeling_count,primary_topic_0921,k052_label,k100_label,k248_label,k351_label,k405_label,k052_majority_primary_topic,k100_majority_primary_topic,k248_majority_primary_topic,k351_majority_primary_topic,k405_majority_primary_topic,rating,version,verdict,quarantine,country_name,users_percent_in_country,ambassador_subreddit,primary_post_language,primary_post_language_percent,secondary_post_language,secondary_post_language_percent,geo_relevant_country_count,geo_relevant_country_codes,geo_relevant_subreddit,table_creation_date,mlflow_run_uuid
603,t5_35ipo,fitnessde,15011,26.0,Fitness and Nutrition,43,74,188,269,312,Fitness and Nutrition,Fitness and Nutrition,Fitness and Nutrition,Fitness and Nutrition,Fitness and Nutrition,pg,None,None,False,Germany,0.861554,False,German,1.000000,None,NaN,1.0,DE,True,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71
444,t5_4ognal,zelten,17047,62.0,Outdoors and Nature,48,89,222,318,367,Place,Music,Outdoors and Nature,Outdoors and Nature,Outdoors and Nature,None,None,None,False,Germany,0.801587,True,German,0.951613,French,0.016129,1.0,DE,True,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71


# Export raw data: 1 row=1 subreddit

Make sure it's ordered by the col to sort subs similar to each other

In [44]:
gspread.__version__

'4.0.1'

In [70]:
# %%time

GSHEET_NAME = 'i18n DE to DE subreddits for OneFeed - model v0.4.0'
GSHEET_KEY = '1FtNYnMW4-VrLcrA_5HwY_qCysqwa089Dh610PCw4p1E'
SH_RAW_SUB_OUTPUT = 'raw_data_per_subreddit'

sh = gc.open_by_key(GSHEET_KEY)
wsh_raw_sub_output = sh.worksheet(SH_RAW_SUB_OUTPUT)


### Note that we have to use `fillna('')`

Otherwise, we'll get errors because the gspread library doesn't know how to handle `pd.NaN` or `np.Nan` (nulls).

In [77]:
%%time

wsh_raw_sub_output.update([df_labels_de.columns.values.tolist()] + 
                          df_labels_de.fillna('').values.tolist())

CPU times: user 38.8 ms, sys: 18 µs, total: 38.8 ms
Wall time: 1.33 s


{'spreadsheetId': '1FtNYnMW4-VrLcrA_5HwY_qCysqwa089Dh610PCw4p1E',
 'updatedCells': 24645,
 'updatedColumns': 31,
 'updatedRange': 'raw_data_per_subreddit!A1:AE795',
 'updatedRows': 795}

### We can read the data back to confirm it's as expected

In [74]:
# Here's how to get the records as a dataframe
pd.DataFrame(wsh_raw_sub_output.get_all_records())

,subreddit_id,subreddit_name,model_distance_order,posts_for_modeling_count,primary_topic_0921,k052_label,k100_label,k248_label,k351_label,k405_label,k052_majority_primary_topic,k100_majority_primary_topic,k248_majority_primary_topic,k351_majority_primary_topic,k405_majority_primary_topic,rating,version,verdict,quarantine,country_name,users_percent_in_country,ambassador_subreddit,primary_post_language,primary_post_language_percent,secondary_post_language,secondary_post_language_percent,geo_relevant_country_count,geo_relevant_country_codes,geo_relevant_subreddit,table_creation_date,mlflow_run_uuid
0,t5_2roop,hardtechno,49,18,,1,1,1,1,1,Music,Music,Music,Music,Music,,,,FALSE,Germany,0.171171,FALSE,English,0.777778,Dutch,0.0555556,1,DE,TRUE,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71
1,t5_2qziu,rappers,76,386,,1,1,1,1,1,Music,Music,Music,Music,Music,r,v2,admin-approved,FALSE,Germany,0.294931,FALSE,English,0.818653,German,0.0181347,1,DE,TRUE,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71
2,t5_2v7pv,germanrap,77,705,Music,1,1,1,1,1,Music,Music,Music,Music,Music,r,v2,,FALSE,Germany,0.851276,FALSE,German,0.714894,English,0.130496,1,DE,TRUE,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71
3,t5_2smd3,musik,85,23,,1,1,1,1,1,Music,Music,Music,Music,Music,,,,FALSE,Germany,0.5125,FALSE,German,0.826087,English,0.0869565,1,DE,TRUE,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71
4,t5_39ea8,mgpmppjwfa,89,25,Music,1,1,1,1,1,Music,Music,Music,Music,Music,,,,FALSE,Germany,0.442478,FALSE,German,0.520000,English,0.08,1,DE,TRUE,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,t5_3oxzh,blaulicht,18989,61,,52,100,247,350,403,"Business, Economics, and Finance",Law,Military,Military,Military,,,,FALSE,Germany,0.891825,FALSE,German,0.950820,English,0.0327869,1,DE,TRUE,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71
790,t5_2ty5z,bundeswehr,18991,254,,52,100,247,350,403,"Business, Economics, and Finance",Law,Military,Military,Military,,,,FALSE,Germany,0.959926,FALSE,German,0.960630,English,0.0314961,1,DE,TRUE,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71
791,t5_2zv9z,recht,19010,60,,52,100,248,351,404,"Business, Economics, and Finance",Law,Law,Law,Law,,,,FALSE,Germany,0.891617,FALSE,German,0.966667,English,0.0166667,1,DE,TRUE,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71
792,t5_1345g1,legaladvicegerman,19015,312,Law,52,100,248,351,404,"Business, Economics, and Finance",Law,Law,Law,Law,r,,,FALSE,Germany,0.866667,FALSE,German,0.714744,English,0.285256,1,DE,TRUE,2021-10-29,134cefe13ae34621a69fcc48c4d5fb71


# Reshape: flatten topics into 1 row = 1 cluster/topic

In [79]:
%%time
col_cluster_level = 'k052_label'
col_cluster_majority_topic = 'k052_majority_primary_topic'

df_cluster_per_row = (
    df_labels_de
    .groupby([col_cluster_level, col_cluster_majority_topic])
    ['subreddit_name']
    .agg(
        [
            ('subreddit_count', 'count'),
            ('list_of_subs', list)
        ]
    )
    .reset_index()
)

# Convert the list of subs into a df & merge back with original sub (each sub should be in a new column)
df_cluster_per_row = (
    df_cluster_per_row
    .merge(
        pd.DataFrame(df_cluster_per_row['list_of_subs'].to_list()).fillna(''),
        how='left',
        left_index=True,
        right_index=True,
    )
    .drop(['list_of_subs'], axis=1)
)

print(df_cluster_per_row.shape)

(48, 86)
CPU times: user 27.7 ms, sys: 0 ns, total: 27.7 ms
Wall time: 27.7 ms


In [80]:
df_cluster_per_row.head()

,k052_label,k052_majority_primary_topic,subreddit_count,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,...,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82
0,1,Music,13,hardtechno,rappers,germanrap,musik,mgpmppjwfa,germusic,kollegah,moneyboy,billytalent,lindemann,eskimocallboy,metallmaimais,avamax,,,,,,,,,,,,,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2,Anime,5,mangade,ueber8000,violaafox,kpopde,terranigma,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,3,Television,5,southparkde,diesimpsons,seriennews,strangerthingsde,lazytown,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,4,"Reading, Writing, and Literature",3,selfpublish_de,buecher,harrypotterde,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,5,Movies,6,filme,streamen,dokumentationen,mediathek,marvelde,kriegdersterne,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [81]:
# df_cluster_per_row.to_csv(
#     f"/content/gdrive/MyDrive/Colab Notebooks/data/{datetime.utcnow().strftime('%Y-%m-%d_%H%M')}_de_to_de_subreddits_raw.csv",
#     index=False,
# )

### by subreddit ID


In [82]:
%%time

col_to_list = 'subreddit_id'
df_cluster_per_row_id = (
    df_labels_de
    .groupby([col_cluster_level, col_cluster_majority_topic])
    [col_to_list]
    .agg(
        [
            ('subreddit_count', 'count'),
            ('list_of_subs', list)
        ]
    )
    .reset_index()
)

# Convert the list of subs into a df & merge back with original sub (each sub should be in a new column)
df_cluster_per_row_id = (
    df_cluster_per_row_id
    .merge(
        pd.DataFrame(df_cluster_per_row_id['list_of_subs'].to_list()).fillna(''),
        how='left',
        left_index=True,
        right_index=True,
    )
    .drop(['list_of_subs'], axis=1)
)

print(df_cluster_per_row_id.shape)

(48, 86)
CPU times: user 33.7 ms, sys: 1.92 ms, total: 35.6 ms
Wall time: 33.6 ms


In [83]:
df_cluster_per_row_id.head()

,k052_label,k052_majority_primary_topic,subreddit_count,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,...,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82
0,1,Music,13,t5_2roop,t5_2qziu,t5_2v7pv,t5_2smd3,t5_39ea8,t5_2t6i4,t5_31l12,t5_2ylk3,t5_2t36v,t5_35q0o,t5_30xre,t5_2tmlgt,t5_nilvp,,,,,,,,,,,,,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2,Anime,5,t5_4thzyd,t5_4sfk6d,t5_50bxag,t5_4rxks9,t5_2w7ha,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,3,Television,5,t5_4r191u,t5_4qwonp,t5_4xjkpg,t5_4r2be4,t5_2sze6,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,4,"Reading, Writing, and Literature",3,t5_4z4tos,t5_2sroz,t5_3jiqq,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,5,Movies,6,t5_2r3zh,t5_4cmjcc,t5_2ti1q,t5_33xyp,t5_1g8x6c,t5_4hb8ta,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [84]:
# df_cluster_per_row_id.to_csv(
#     f"/content/gdrive/MyDrive/Colab Notebooks/data/{datetime.utcnow().strftime('%Y-%m-%d_%H%M')}_de_to_de_subreddits_raw_ids.csv",
#     index=False,
# )

## Add partial list of subreddits to filter out

Most of the NSFW subredits are in these clusters:

But some were mis-classified

In [76]:
l_clusters_to_remove = [
    # NSFW clusters (porn/celebs)
    6,
    7,
    8,
    9,
    46,  # Sexual orientation & NSFW

    # drinking & drugs
    39,  
    44,  # drugs and detoxing?
]
l_subs_manual_remove = [
    'sexmeets1',
    'fuck',
    'eastgermandreams',
    'BonnyLangOfficial',

    # potential misinformation
    'wuhan_virus',
]

# Subs that appear to be misclassified, check to see what we can learn to improve
l_subs_investigate = [
    'outdoor',  # classified in podcast group
    
    'satire_de_en', # satire is hard to classify...
]